# Let's Build Personalized Website using Agentic Workflows and Inferentia

Notes: 
1. Use the native Jupyter Notebook to leverage the full features of this notebook

## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import base64
import boto3
import io
import os
import json
import re
import importlib
import shutil
import re
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.core.display import display, HTML
from PIL import Image

In [ ]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

## Useful functions
Let's define some functions that'll be used in the process of gathering the content for the website.

In [ ]:
def build_profile(UserProfile):
    profile = f"Your customer is {UserProfile['Name']} located in {UserProfile['Location']}. They are in {UserProfile['Industry']} industry. Their mission statement: `{UserProfile['Mission']}.'"
    return profile

In [ ]:
def get_labels_and_desc(pattern, text):  
    matches = re.findall(pattern, text)
    myList = [] #(label, description)
    for match in matches:
        title = match[0]
        description = match[1]
        myList.append((title,description))
    return myList

In [ ]:
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = io.BytesIO(base64_image)
  return Image.open(buffer)
 
# Display PIL images as grid
def display_image(image=None,width=500,height=500):
    img = image.resize((width, height))
    display(img)

## Define variables with SageMaker Model endpoints

In [ ]:
model_llama3p1_endpoint = 'nous-research-Meta-Llama-3-8B-2024-10-12-22-52-51-435-endpoint'
model_stablediffV1_endpoint = 'huggingface-pytorch-inference-neuronx-2024-10-10-20-28-26-629'

### Check the status of the endpoints

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_llama3p1_endpoint)
print(f"Status for {model_llama3p1_endpoint}: {resp['EndpointStatus']}")

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=model_stablediffV1_endpoint)
print(f"Status for {model_stablediffV1_endpoint}: {resp['EndpointStatus']}")

## Example User Profiles
Define a few sample profiles. These profiles define an Industry, a name for the company, location and a mission statement for the company.

In [ ]:
Profiles = {
    'Construction-Example': {
        'Name': 'Example Corp Construction Inc',
        'Industry': 'Construction',
        'Location': 'New York City, NY',
        'Mission': 'Building a sustainable future for New York'
    },
    'Manufacturing-Example': {
        'Name': 'Example Corp Manuf LLC',
        'Industry': 'Manufacturing',
        'Location': 'San Jose, CA',
        'Mission': 'Building the next generation Electric Vehicles'
    },
    'Mining-Example': {
        'Name': 'Example Corp Mining Inc',
        'Industry': 'Mining',
        'Location': 'Bisbee, AZ',
        'Mission': 'Extracting the value for America'
    }
}

### Define Inputs (content) for the website

The homepage of a company often has it's offerings, few testimonials from customers, icons, etc. Let's now define some base content for these.

Let's start with choosing an industry for your company.

In [ ]:
dropdown = widgets.Dropdown(
       options=['Manufacturing', 'Construction', 'Mining'],
       value='Manufacturing',
       description='Select your example industry:',
   )

### Select an Industry for your company

In [ ]:
display(dropdown)

### Offerings

Read some pre-defined offerings for each Industry. In practice, this will be a document that has the offerings of your company.

In [ ]:
with open("./references/offerings.json", "r") as file:
    offerings = json.load(file)

In [ ]:
customer= Profiles[dropdown.value+'-Example']
offering = re.sub(r'\n#?', ' ', offerings[customer['Industry']])
project = dropdown.value + "_Website_Test"

In [ ]:
print(f"Selected Customer:\n{customer}")

#### Natural Language description of customer profile

In [ ]:
customer['Description'] = build_profile(customer)
print(customer['Description'])

### Testimonials
Read some pre-defined testimonials for each Industry.

In [ ]:
with open('./references/Testimonials.json', 'r') as file:
    testimonials = json.load(file)

### Retrieve painpoints of a customer industry
We now read a file corresponding to the pain-points and challenges corresponding to the industry you have chosen above. In a real use case, you might have a collection of documents with these details and you can leverage RAG to get a blurb of challenges that are relevant for your company.

In this scenario, we just read the entire file that has multiple challenges.

In [ ]:
query = f"{customer['Description']}\nList the industry pain-points and challenges"
print(f"Here is your customer:\n{query}")

In [ ]:
with open('references/KnowledgeBase/Manufacturing-Industry-DummyArticle-01.txt', 'r') as file:
    contexts_painpoints = file.read()

print(f"These are the applicable pain-points/challenges:\n{contexts_painpoints}")

## Prompting AI to describe a website personalized for this profile and background

We now get to the crux of the application where we leverage LLMs to create the content for the website.

Here the design of the prompt is critical. Please see the guidance provided to the LLM in the below prompt. We also ask the response to be in two parts: 
1. Description
2. Visual Elements.

### Finding the possible painpoints using a LLM

We now create a prompt with the 'painpoints' from above as context. Please ensure that the number of tokens in the context (and prompt) <= 512 + sequence length with which the llama 3.1 8B model was compiled for. Else, you'll run into an error.

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Research Document ***
{contexts_painpoints}
*** End of Research Document ***


Question: {customer['Description']} Based on the above research, list top-3 challenges and explain we understand their challanges. Do not use any company name.
Provide your response in bullet points without any preamble (e.g. • a pain-point: one sentence description, • pain-point: one sentence description, etc.). 

"""
params = {"max_new_tokens": 512, "do_sample": False}

Invoke the LLM

In [ ]:
%%time
response_model= smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_painpoints = json.load(response_model['Body'])
top_pain_points_text= response_painpoints['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top painpoints:\n{top_pain_points_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
top_painpoints = get_labels_and_desc(pattern, top_pain_points_text)
[print(f"{key}: {value}\n")for (key, value) in top_painpoints]

### Finding the personalized offerings using LLM

We now use the pre-defined offerings as context to generate content corresponding to top offerings that will be advertised on the website.

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an experienced business consultant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
*** Offerings Document ***
{offering}
*** Offerings Document ***


Question: {customer['Description']} Their top-3 painpoints are {top_painpoints}.

List and briefly advertise the top-3 offerings for {customer['Name']} based on the offerings document above.
Provide your list in bullet points without any preamble (e.g. • an offering: short description, • an offering: short description, ...). 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
params = {"max_new_tokens": 384, "do_sample": False}

Invoke the model.

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_offerings = json.load(response_model['Body'])
top_offerings_text= response_offerings['generated_text'].split('<|eot_id|>')[0].replace('\n',', ')
print(f"Raw Output - top offerings:\n{top_offerings_text}\n")

In [ ]:
# Convert this into a function
pattern = r'•\s*(.*?):\s*(.*?)\.'
top_offerings = get_labels_and_desc(pattern, top_offerings_text)
[print(f"{key}: {value}\n")for (key, value) in top_offerings]

### Art Design

We now create a prompt that is used to generate descriptions for images and icons that will be embedded in the website. Note that the prompt asks the model to generate the output in a structured format.

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a UI/UX art designer personalizing visual website content such as hero image and icons tailored to customer profile, personalized offerings and pain-points. 
Your task is to describe a hero image and six icons. For the Hero Image, design an attention-grabbing image that relates to the customer's industry, location, and mission. For icons, design simple, memorable icons that visually represent the concepts.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

For the Hero Image:
- Describe a scene that represents the customer's industry and mission
- Include elements that reflect the customer location
- Mention 1-2 key visual elements

For each icon:
- Describe a simple, recognizable shape or symbol
- Concisely describe the key visual element
- Call our that this is an 'icon'

Each icon should be distinct yet part of a cohesive set in style and color.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide your response in the following structured format:

** Hero Image[Description in 1-2 sentences]**
** {top_painpoints[0][0]} Icon [Description in one sentence]**
** {top_painpoints[1][0]} Icon [Description in one sentence]**
** {top_painpoints[2][0]} Icon [Description in one sentence]**
** {top_offerings[0][0]} Icon [Description in one sentence]**
** {top_offerings[1][0]} Icon [Description in one sentence]**
** {top_offerings[2][0]} Icon [Description in one sentence]**

Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

Invoke the model.

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_art= json.load(response_model['Body'])
art_descriptions_text = response_art['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**,','**')
print(f"Raw Output - top offerings:\n{art_descriptions_text}\n")

In [ ]:
# Creating the variable for the art descriptions that will help us to create a better prompt later
art_descriptions = art_descriptions_text.split("**")[1:]
art_descriptions = {art_descriptions[i].strip():{"Description":art_descriptions[i+1].strip(), \
                    'FileName':art_descriptions[i].replace(' ','')+'.jpg'} for i in range(0,int(len(art_descriptions)),2)}

In [ ]:
print(f"Here is the art and their descriptions:\n {json.dumps(art_descriptions, indent=4)}")

### Call to action, hero labels and description - final touches

Use the LLM one final time to generate a catchy slogan and call-to-action.

In [ ]:
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are creating engaging website content for AnyCompany Consulting, which is a business consulting company. 
Your task is to generate four specific pieces of text based on the customer profile and offerings provided.

Customer Profile:
- Industry: {customer['Industry']}
- Location: {customer['Location']}
- Company Motto/Mission: {customer['Mission']}

Hero image: {art_descriptions['Hero Image']['Description']}

Industry painpoints:
- {top_painpoints[0][0]}
- {top_painpoints[1][0]}
- {top_painpoints[2][0]}

AnyCompany Consulting offerings:
- {top_offerings[0][0]}
- {top_offerings[1][0]}
- {top_offerings[2][0]}

<|eot_id|><|start_header_id|>user<|end_header_id|>

Generate the following four pieces of text:
1. Hero Label: A short, attention-grabbing phrase to work with AnyCompany Consulting (5-7 words)
2. Hero Text: A brief explanation expanding on the label (15-20 words)
3. Call-to-Action Button: A short, action-oriented phrase (3-5 words)
4. Call-to-Action Explanation: A motivating statement to encourage engagement with AnyCompany Consulting (15-20 words)

Use this exact format for your response:

** Hero Label **
** Hero Text **
** Call-to-Action Button **
** Call-to-Action Explanation **

Ensure each piece of text relates to the customer's industry, pain points, and the consulting offerings. 
Focus on engaging the customer and encouraging them to work with AnyCompany Consulting.
Provide your response using the exact format above, without any preamble.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
params = {"max_new_tokens": 512, "do_sample": False}

Invoke the model.

In [ ]:
%%time
response_model = smr_client.invoke_endpoint(
    EndpointName=model_llama3p1_endpoint,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)
response_text= json.load(response_model['Body'])
text_descriptions_text = response_text['generated_text'].split('<|eot_id|>')[0]\
                        .replace('\n',', ').replace(', ,', '').replace('**, ','**')
print(f"Raw Output - Description of critical text:\n{text_descriptions_text}\n")

In [ ]:
text_descriptions_lst = text_descriptions_text.split('**')
text_descriptions_lst = [x for x in text_descriptions_lst if x is not '']
text_descriptions = {"HeroLabel":text_descriptions_lst[0], "HeroText":text_descriptions_lst[1], "CTALabel":text_descriptions_lst[2] ,"CTAText":text_descriptions_lst[3]}

In [ ]:
print(text_descriptions)

### Image Creation based on Image Descriptions

We now create images and icons for the website using Stable Diffusion. The descriptions that were generated in the previous step are used as prompts to the Stable Diffusion model. We successively call the APIs and then save the files.

In [ ]:
%%time
# WE HAVE TO OPTIMIZE THIS FOR ICONS.
from sagemaker.huggingface.model import HuggingFacePredictor
predictor_SDv1 = HuggingFacePredictor(model_stablediffV1_endpoint)

images = []
image_files = []
for art, content in art_descriptions.items():
    if 'Icon' in art:
        inf_steps = 30
    else:
        inf_steps = 50

    params = {
    "num_inference_steps" : inf_steps,
    "negative_prompt" : "disfigured, ugly, deformed, unprofessional"
    }
    
    prompt = content['Description']
    response = predictor_SDv1.predict(data={"inputs": prompt,"parameters":params})

    image = decode_base64_image(response["generated_images"][0])
    image_files.append(content['FileName'])

    # Save the image
    image.save(f"{content['FileName']}")
    images.append(np.array(image))

In [ ]:
print("Number of visual assets created: {}".format(len(image_files)))

In [ ]:
shutil.copy("./references/anycompany_logo.jpg", "./anycompany_logo.jpg")

Let's plot the images generated

In [ ]:
# Plot the images
fig, axs = plt.subplots(1, len(images), figsize=(30, 20))

for i, (image, ax, filename) in enumerate(zip(images, axs, image_files)):
    ax.imshow(image)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

### Let's Generate our HTML file

The html file combines the images and text generated above to create the website. You can also consider using a code generation model to generate the html file. For the purpose of this session, we use an existing html template.

In [ ]:
CSS_section = open('./references/CSS.txt', 'r').read()
CSS_section = CSS_section.replace("hero-image.jpg", art_descriptions['Hero Image']['FileName'])
JS_section = open('./references/JS.txt', 'r').read()
full_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>AnyCompany Consulting Personalized Page</title>
  <style>
{CSS_section}
  </style>
</head>
<body>
  <header>
    <div class="logo">
      <img src="anycompany_logo.jpg" alt="AnyCompany Consulting">
    </div>
    <nav>
      <ul>
        <li><a href="#">Home</a></li>
        <li><a href="#">About Us</a></li>
        <li><a href="#">Services</a></li>
        <li><a href="#">Projects</a></li>
        <li><a href="#">Resources</a></li>
        <li><a href="#">Contact</a></li>
      </ul>
    </nav>
  </header>
  <main>
    <section class="hero">
      <h1>{text_descriptions['HeroLabel']}</h1>
      <p>{text_descriptions['HeroText']}</p>
    </section>
    <section class="offerings">
      <h2>Our Tailored Solutions for Your {customer['Industry']} Needs</h2>
      <div class="offerings-grid">
        <div class="offering-card">
          <img src="{art_descriptions[top_offerings[0][0]+' Icon']['FileName']}" alt="{top_offerings[0][0]}">
          <h3>{top_offerings[0][0]}</h3>
          <p>{top_offerings[0][1]}</p>
        </div>
        <div class="offering-card">
          <img src="{art_descriptions[top_offerings[1][0]+' Icon']['FileName']}" alt="{top_offerings[1][0]}">
          <h3>{top_offerings[1][0]}</h3>
          <p>{top_offerings[1][1]}</p>
        </div>
        <div class="offering-card">
          <img src="{art_descriptions[top_offerings[2][0]+' Icon']['FileName']}" alt="{top_offerings[2][0]}">
          <h3>{top_offerings[2][0]}</h3>
          <p>{top_offerings[2][1]}</p>
        </div>
      </div>
    </section>
    <section class="pain-points">
      <h2>Addressing Your {customer['Industry']} Challenges</h2>
      <div class="pain-points-grid">
        <div class="pain-point-card">
          <img src="{art_descriptions[top_painpoints[0][0]+' Icon']['FileName']}" alt="{top_painpoints[0][0]}">
          <h3>{top_painpoints[0][0]}</h3>
          <p>{top_painpoints[0][1]}</p>
        </div>
        <div class="pain-point-card">
          <img src="{art_descriptions[top_painpoints[1][0]+' Icon']['FileName']}" alt="{top_painpoints[1][0]}">
          <h3>{top_painpoints[1][0]}</h3>
          <p>{top_painpoints[1][1]}</p>
        </div>
        <div class="pain-point-card">
          <img src="{art_descriptions[top_painpoints[2][0]+' Icon']['FileName']}" alt="{top_painpoints[2][0]}">
          <h3>{top_painpoints[2][0]}</h3>
          <p>{top_painpoints[2][1]}</p>
        </div>
      </div>
    </section>
    <section class="testimonials">
      <h2>What Our Clients Say</h2>
      <div class="testimonial-container">
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][0]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][0]['author']}, {testimonials['testimonials'][0]['position']} from {testimonials['testimonials'][0]['company']}</p>
        </div>
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][1]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][1]['author']}, {testimonials['testimonials'][1]['position']} from {testimonials['testimonials'][1]['company']}</p>
        </div>
        <div class="testimonial" style="display: none;">
          <p class="testimonial-text">{testimonials['testimonials'][2]['text']}</p>
          <p class="testimonial-author">- {testimonials['testimonials'][2]['author']}, {testimonials['testimonials'][2]['position']} from {testimonials['testimonials'][2]['company']}</p>
        </div>
      </div>
    </section>
    <section class="cta">
      <h2>{text_descriptions['CTALabel']}</h2>
      <p>{text_descriptions['CTAText']}</p>
      <a href="#" class="cta-button">Get Started</a>
    </section>
  </main>
  <footer>
    <p>&copy; 2024 AnyCompany Consulting. All rights reserved.</p>
    <p>
      <a href="#">Address</a> | <a href="#">Phone</a> | <a href="#">Email</a>
    </p>
  </footer>
  <script>
  {JS_section}
  </script>
</body>
</html>
"""

### Putting the assets into a single folder

Create the folder for the project

In [ ]:
try:
    os.mkdir(project)
    print(f"Project Folder: {project}" + '_reInvent')
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from IPython.display import HTML

In [ ]:
with open('main.html', 'w', encoding='utf-8') as file:
    file.write(full_html)

In [ ]:
project_files = image_files.copy()
project_files.append('main.html')
project_files.append('anycompany_logo.jpg')

Copy the generated files and the html file to the project folder.

In [ ]:
for file in project_files:
    destination = project + "/" + file
    shutil.move(file, destination)

## You can now download the folder and view the html in a browser. You have successfully generated a website using Generative AI !!